In [3]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

ModuleNotFoundError: No module named 'alpaca.markets'

In [15]:
def fetch_data_from_api(url, params=None):
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.text}")
        return {}

def get_quotes(symbol, api_key):
    quote_url = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(quote_url)

def get_financials_reported(symbol, api_key):
    reported_financials_url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(reported_financials_url)

def get_usa_spending(symbol, api_key):
    spending_url = f"https://finnhub.io/api/v1/stock/usa-spending?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(spending_url)

def get_company_basic_financials(symbol, api_key):
    basic_financials_url = f"https://finnhub.io/api/v1/stock/metric?symbol={symbol}&metric=all&token={api_key}"
    return fetch_data_from_api(basic_financials_url)

def get_company_profile(symbol, api_key):
    profile_url = f"https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(profile_url)

In [16]:
def extract_financial_data(data, section, concepts=None):
    financial_data = []
    if 'data' in data:
        for report in data['data']:
            if 'report' in report and section in report['report']:
                section_data = report['report'][section]
                if concepts:  # Filter data based on the list of important concepts
                    section_data = [item for item in section_data if item['concept'] in concepts]
                financial_data.extend(section_data)
    return financial_data


In [17]:
# Function to process and flatten basic financials data
def process_basic_financials_data(data, symbol):
    if 'metric' not in data:
        print(f"No 'metric' data available for {symbol}.")
        return pd.DataFrame()

    metrics = data['metric']
    relevant_keys = [
        'quickRatioAnnual', 'netProfitMarginAnnual', 'inventoryTurnoverAnnual', 'grossMarginAnnual', 'totalDebt/totalEquityAnnual',
        'assetTurnoverAnnual', 'receivablesTurnoverAnnual', 'roiAnnual', 'cashFlowPerShareAnnual', 'ebitdPerShareAnnual'
    ]
    relevant_metrics = {k: metrics.get(k) for k in relevant_keys}
    relevant_metrics['symbol'] = symbol  # Add the symbol to distinguish between companies
    return pd.DataFrame([relevant_metrics])

In [18]:
# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')  # Your RDS username
password = os.getenv('password')  # Your RDS password
host = os.getenv('host')  # Your RDS endpoint
port = os.getenv('port')  # Default MySQL port
database = os.getenv('database')  # The name of the database you want to create

api_key = os.getenv('FINNHUB_API_KEY')  # Replace with your actual API key

# Create an engine to connect to MySQL Database using a hardcoded URL for testing
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=364)
formatted_start_date = datetime.strptime(start_date.strftime('%Y-%m-%d'), '%Y-%m-%d')
formatted_end_date = datetime.strptime(end_date.strftime('%Y-%m-%d'), '%Y-%m-%d')

company_symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]
concepts = ['us-gaap_AssetsCurrent', 'us-gaap_AssetsNoncurrent', 'us-gaap_Assets',
            'us-gaap_LiabilitiesCurrent', 'us-gaap_LiabilitiesNoncurrent', 'us-gaap_Liabilities',
            'us-gaap_StockholdersEquity', 'us-gaap_CommonStocksIncludingAdditionalPaidInCapital',
            'us-gaap_RetainedEarningsAccumulatedDeficit']

quotes_data = []
financials_reported_data = []
spending_data = []
basic_financials_data = []
symbols_data = []

for symbol in company_symbols:
    profile_data = get_company_profile(symbol, api_key)
    # Assuming that the profile API returns a 'name' field for the description
    description = profile_data.get('name', 'N/A')  # Use 'description' if it's available
    symbols_data.append({'symbol': symbol, 'description': description})
    time.sleep(1)  # Respect API rate limits

    # Fetch and process quotes
    quote = get_quotes(symbol, api_key)
    if quote:
        quote['symbol'] = symbol
        quotes_data.append(quote)

    # Fetch and process financials
    financial_data = get_financials_reported(symbol, api_key)
    bs_data = extract_financial_data(financial_data, 'bs', concepts)
    for item in bs_data:
        item['symbol'] = symbol
    financials_reported_data.extend(bs_data)

    # Fetch and process spending data
    spend_data = get_usa_spending(symbol, api_key)
    if 'data' in spend_data:
        for spend in spend_data['data']:
            spend_date = pd.to_datetime(spend['actionDate'])
            if formatted_start_date <= spend_date <= formatted_end_date:
                spend['symbol'] = symbol
                spending_data.append(spend)

    basic_financials_response = get_company_basic_financials(symbol, api_key)
    if basic_financials_response:
        processed_data = process_basic_financials_data(basic_financials_response, symbol)
        basic_financials_data.append(processed_data)

    time.sleep(1)  # Respect API rate limits

# Convert lists to DataFrames
symbols_df = pd.DataFrame(symbols_data)
quotes_df = pd.DataFrame(quotes_data)
financials_reported_df = pd.DataFrame(financials_reported_data)
spending_df = pd.DataFrame(spending_data)
basic_financials_with_median_df = pd.concat(basic_financials_data, ignore_index=True)
symbols_df = pd.DataFrame({'symbol': company_symbols})

# Handling missing values by imputing with median, excluding 'symbol'
numerical_cols = basic_financials_with_median_df.columns.drop('symbol')  # Adjust the column list as needed
medians = basic_financials_with_median_df[numerical_cols].median()
basic_financials_with_median_df[numerical_cols] = basic_financials_with_median_df[numerical_cols].fillna(medians)

# Upload each DataFrame to the MySQL database
symbols_df.to_sql(name='symbols', con=engine, if_exists='replace', index=False)
quotes_df.to_sql(name='quotes', con=engine, if_exists='replace', index=False)
financials_reported_df.to_sql(name='financials_reported', con=engine, if_exists='replace', index=False)
spending_df.to_sql(name='usa_spending', con=engine, if_exists='replace', index=False)
basic_financials_with_median_df.to_sql(name='basic_financials', con=engine, if_exists='replace', index=False)

print("Data uploaded successfully.")

Data uploaded successfully.


In [19]:
spending_df

,symbol,recipientName,recipientParentName,country,totalValue,actionDate,performanceStartDate,performanceEndDate,awardingAgencyName,awardingSubAgencyName,awardingOfficeName,performanceCountry,performanceCity,performanceCounty,performanceState,performanceZipCode,performanceCongressionalDistrict,awardDescription,naicsCode,permalink
0,JPM,J. P. MORGAN INVESTMENT MANAGEMENT INC.,JPMORGAN CHASE & CO.,USA,5018724.5,2023-09-27,2015-09-01,2024-08-31,Pension Benefit Guaranty Corporation,Pension Benefit Guaranty Corporation,PENSION BENEFIT GUARANTY CORP,USA,NEW YORK,NEW YORK,36,NEW YORK,101670001,IGF::CT::IGF,523930,https://www.usaspending.gov/award/CONT_AWD_PBG...
1,JPM,J. P. MORGAN INVESTMENT MANAGEMENT INC.,JPMORGAN CHASE & CO.,USA,4291692.5,2023-09-25,2015-09-01,2024-08-31,Pension Benefit Guaranty Corporation,Pension Benefit Guaranty Corporation,PENSION BENEFIT GUARANTY CORP,USA,NEW YORK,NEW YORK,36,NEW YORK,101670001,IGF::CT::IGF,523991,https://www.usaspending.gov/award/CONT_AWD_PBG...
2,JPM,J. P. MORGAN INVESTMENT MANAGEMENT INC.,JPMORGAN CHASE & CO.,USA,5018724.5,2023-08-09,2015-09-01,2024-08-31,Pension Benefit Guaranty Corporation,Pension Benefit Guaranty Corporation,PENSION BENEFIT GUARANTY CORP,USA,NEW YORK,NEW YORK,36,NEW YORK,100160450,IGF::CT::IGF,523930,https://www.usaspending.gov/award/CONT_AWD_PBG...
3,JPM,J. P. MORGAN INVESTMENT MANAGEMENT INC.,JPMORGAN CHASE & CO.,USA,4291692.5,2023-08-08,2015-09-01,2024-08-31,Pension Benefit Guaranty Corporation,Pension Benefit Guaranty Corporation,PENSION BENEFIT GUARANTY CORP,USA,NEW YORK,NEW YORK,36,NEW YORK,101670001,IGF::CT::IGF,523991,https://www.usaspending.gov/award/CONT_AWD_PBG...
4,JPM,J. P. MORGAN INVESTMENT MANAGEMENT INC.,JPMORGAN CHASE & CO.,USA,4291692.5,2023-07-24,2015-09-01,2023-08-31,Pension Benefit Guaranty Corporation,Pension Benefit Guaranty Corporation,PENSION BENEFIT GUARANTY CORP,USA,NEW YORK,NEW YORK,36,NEW YORK,101670001,IGF::CT::IGF,523991,https://www.usaspending.gov/award/CONT_AWD_PBG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,MRK,MERCK SHARP & DOHME LLC,"MERCK & CO., INC.",USA,0.0,2023-05-22,2018-09-01,,Department of Veterans Affairs,Department of Veterans Affairs,NAC FEDERAL SUPPLY SCHEDULE (36F797),,,,,,,"65 I B, DRUGS, PHARMACEUTICALS,&HEMATOLOGY REL...",325412,https://www.usaspending.gov/award/CONT_IDV_36F...
3707,MRK,BIOMARK LLC,"MERCK & CO., INC.",USA,304126.1,2023-05-17,2023-05-17,2023-09-30,Department of the Interior,Bureau of Reclamation,DENVER FED CENTER,USA,KLAMATH FALLS,KLAMATH,41,OREGON,976039365,"PASSIVE INTEGRATED TRANSPONDERS (PIT) TAGS, RE...",334511,https://www.usaspending.gov/award/CONT_AWD_140...
3708,MRK,MERCK SHARP & DOHME LLC,"MERCK & CO., INC.",USA,3840008.8,2023-05-17,2021-09-16,2026-09-15,Department of Health and Human Services,Centers for Disease Control and Prevention,CDC OFFICE OF ACQUISITION SERVICES,USA,KENILWORTH,UNION,34,NEW JERSEY,070331310,"VACCINE STORAGE AND ROTATION, 2021-2026",493120,https://www.usaspending.gov/award/CONT_AWD_75D...
3709,MRK,ORGANON LLC,ORGANON LLC,USA,0.0,2023-05-08,2023-09-01,,Department of Veterans Affairs,Department of Veterans Affairs,NAC PHARMACEUTICALS (36E797),,,,,,,NEW AWARD FOR RENFLEXIS (INFLIXIMAB-ABDA) 100M...,325412,https://www.usaspending.gov/award/CONT_IDV_36E...


In [21]:
financials_reported_df

,concept,unit,label,value,symbol
0,us-gaap_Assets,usd,Total assets,3.875393e+12,JPM
1,us-gaap_Liabilities,usd,Total liabilities,3.547515e+12,JPM
2,us-gaap_RetainedEarningsAccumulatedDeficit,usd,Retained earnings,3.329010e+11,JPM
3,us-gaap_StockholdersEquity,usd,Total stockholders’ equity,3.278780e+11,JPM
4,us-gaap_Assets,usd,Total assets,3.665743e+12,JPM
...,...,...,...,...,...
722,us-gaap_Assets,usd,Total Assets,1.057810e+11,MRK
723,us-gaap_AssetsCurrent,usd,Total current assets,2.906400e+10,MRK
724,us-gaap_LiabilitiesCurrent,usd,Total current liabilities,1.564100e+10,MRK
725,us-gaap_RetainedEarningsAccumulatedDeficit,usd,Retained Earnings (Accumulated Deficit),3.753600e+10,MRK


In [22]:
quotes_df

,c,d,dp,h,l,o,pc,t,symbol
0,193.49,0.12,0.0621,194.8700,193.06,193.57,193.37,1714161602,JPM
1,427.57,7.52,1.7903,428.5300,419.64,420.23,420.05,1714161602,GS
2,62.66,0.87,1.4080,63.2250,61.54,61.51,61.79,1714161601,C
3,182.50,1.99,1.1024,184.1000,181.76,180.63,180.51,1714161602,JLL
4,112.73,-0.04,-0.0355,113.0218,111.32,111.85,112.77,1714161748,DIS
5,40.05,0.66,1.6756,40.3800,39.41,39.54,39.39,1714161602,TPR
6,12.79,-0.25,-1.9172,13.0200,12.61,12.76,13.04,1714161646,F
7,117.96,-3.37,-2.7775,119.1000,116.22,119.00,121.33,1714161742,XOM
8,169.30,-0.59,-0.3473,171.3400,169.19,169.87,169.89,1714161601,AAPL
9,179.62,5.95,3.4260,180.8200,176.13,177.68,173.67,1714161601,AMZN


In [23]:
basic_financials_with_median_df

,quickRatioAnnual,netProfitMarginAnnual,inventoryTurnoverAnnual,grossMarginAnnual,totalDebt/totalEquityAnnual,assetTurnoverAnnual,receivablesTurnoverAnnual,roiAnnual,cashFlowPerShareAnnual,ebitdPerShareAnnual,symbol
0,0.9824,29.27542,9.13935,45.555,1.8660,0.76705,9.5919,5.27,4.5101,23.4868,JPM
1,0.3568,7.85000,9.13935,37.410,9.1510,0.06600,0.8093,0.72,-46.0856,45.0983,GS
2,0.9824,20.13062,9.13935,45.555,2.8361,0.76705,9.5919,1.17,-42.0358,8.9324,C
3,0.9757,1.09000,9.13935,57.040,0.3496,1.29230,8.3999,2.65,8.1857,17.1968,JLL
4,0.9891,2.65000,31.95740,33.410,0.4701,0.43240,8.5821,1.61,2.7496,5.6809,DIS
5,1.0243,14.05000,2.03380,70.780,0.7302,0.93590,28.7232,23.75,3.4785,5.6067,TPR
6,1.0423,2.47000,10.76530,9.170,3.4889,0.64470,11.0774,2.26,1.6074,2.7691,F
7,1.0945,10.76000,9.27910,31.310,0.2036,0.88940,10.6017,14.61,8.4236,16.7542,XOM
8,0.9444,25.31000,37.97770,44.130,1.8040,1.08710,13.2873,55.66,6.4041,7.9326,AAPL
9,0.8103,5.29000,8.99960,46.980,0.4149,1.08890,13.0104,10.65,3.1029,3.5691,AMZN


In [24]:
# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')  # Your RDS username
password = os.getenv('password')  # Your RDS password
host = os.getenv('host')  # Your RDS endpoint
port = os.getenv('port')  # Default MySQL port
database = os.getenv('database')  # The name of the database you want to create

# Connect to the database without specifying the database name first
# This checks if your instance connection is fine
test_engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}')



try:
    with test_engine.connect() as test_connection:
        print("RDS instance connection successful!")
except Exception as e:
    print(f"Failed to connect to RDS instance: {e}")

# If the instance connection is successful, connect to your specific database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

try:
    with engine.connect() as connection:
        print(f"Connected successfully to the database '{database}'!")
except Exception as e:
    print(f"Failed to connect to the database '{database}': {e}")



RDS instance connection successful!
Connected successfully to the database 'mayah_bosworth_sql_project'!


In [6]:
# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')  # Your RDS username
password = os.getenv('password')  # Your RDS password
host = os.getenv('host')  # Your RDS endpoint
port = os.getenv('port')  # Default MySQL port
database = os.getenv('database')  # The name of the database you want to create

api_key = os.getenv('FINNHUB_API_KEY')  # Replace with your actual API key

# Create an engine to connect to MySQL Database using a hardcoded URL for testing
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

company_symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]
symbols_df = pd.DataFrame({'symbol': company_symbols})
symbols_df.to_sql(name='symbols', con=engine, if_exists='replace', index=False)

12

In [8]:
import os
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import requests  # Make sure to import the requests module

# Existing API fetching functions...

def fetch_data_from_api(url, params=None):
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.text}")
        return {}

# ... Other fetching functions ...

# New function to get the company profile including the symbol and its description
def get_company_profile(symbol, api_key):
    profile_url = f"https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(profile_url)

# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

# Your Finnhub API key
api_key = os.getenv('FINNHUB_API_KEY')

# Create an engine to connect to MySQL Database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

company_symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]

# Now, let's gather the symbols and descriptions
symbols_data = []
for symbol in company_symbols:
    profile_data = get_company_profile(symbol, api_key)
    # Assuming that the profile API returns a 'name' field for the description
    description = profile_data.get('name', 'N/A')  # Use 'description' if it's available
    symbols_data.append({'symbol': symbol, 'description': description})
    time.sleep(1)  # Respect API rate limits

# Convert symbols_data list to DataFrame
symbols_df = pd.DataFrame(symbols_data)

# Upload the symbols DataFrame to the MySQL database
symbols_df.to_sql(name='symbols', con=engine, if_exists='replace', index=False)

print("Symbols data uploaded successfully.")


Symbols data uploaded successfully.
